# Final Project: License Plate Reader


Manual license plate reading is prone to errors, especially when there are multiple vehicles, or the plates are dirty, obstructed or damaged. Automated license plate reading (ALPR) using computer vision can improve accuracy, speed up the process, and reduce human error. By using methodologies that we used in class such as image processing/transformation applications and feature matching, we should be able to have our code correctly identify the edges and boundaries of each character to successfully read a license plate.

In order to achieve this, we will be going through the following process:
- Be able to identify the placement of the license plate.
- Identify each character on the license plate.
- Classify each character and add up to get the full number.

## Identifying the License Plate

In [3]:
import cv2
import numpy as np
from keras_ocr.detection import Detector
from keras_ocr.recognition import Recognizer
from PIL import Image

def load_image(filepath):
    img = Image.open(filepath)
    return (np.asarray(img).astype(float)/255)[:, :, :3]



In [2]:
img = load_image("Cars147.png")
img = cv2.resize(img, (600, 400))

detector = Detector()
plate = detector.detect([img])[0][0]

recognizer = Recognizer()
img = np.asarray(img)
characters = recognizer.recognize([img])

plate_number = ''.join(characters)

print(plate_number)

Looking for C:\Users\mario\.keras-ocr\craft_mlt_25k.h5
1/1 [==============================] - 1s 1s/step


IndexError: index 0 is out of bounds for axis 0 with size 0

In [4]:
# Import necessary libraries
import cv2
import easyocr

# Load the image of the license plate using OpenCV
img = cv2.imread('Cars248.png')

# Resize the image to a fixed width while maintaining the aspect ratio
img_width = 600
aspect_ratio = img.shape[1] / img.shape[0]
img_height = int(img_width / aspect_ratio)
img = cv2.resize(img, (img_width, img_height))

# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply Gaussian blur to reduce noise and smooth the image
kernel_size = (5, 5)
blur = cv2.GaussianBlur(gray, kernel_size, 0)

# Apply adaptive thresholding to separate the foreground (license plate) from the background
max_value = 255
block_size = 19
constant = 9
th = cv2.adaptiveThreshold(blur, max_value, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, block_size, constant)

# Apply edge detection to highlight the edges of the license plate
canny = cv2.Canny(th, 30, 150)

# Display the image with the license plate region highlighted
cv2.imshow('License Plate Detection', canny)
cv2.waitKey(0)
cv2.destroyAllWindows()


# Find contours in the image
contours, hierarchy = cv2.findContours(canny, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

min_area = 10000

# Create an EasyOCR reader object
reader = easyocr.Reader(['en'])

# Iterate over each contour to find the license plate
for contour in contours:
    # Get the bounding rectangle of the contour
    x, y, w, h = cv2.boundingRect(contour)
    aspect_ratio = w / h
    
    # Check if the aspect ratio of the bounding rectangle is within a certain range to filter out non-license plate regions
    min_aspect_ratio = 3
    max_aspect_ratio = 4
    if aspect_ratio >= min_aspect_ratio and aspect_ratio <= max_aspect_ratio and w*h >= min_area :
        # Draw a rectangle around the license plate region
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # Extract the license plate region from the image
        plate_img = img[y:y + h, x:x + w]

        # Use EasyOCR to recognize the text in the license plate region
        result = reader.readtext(plate_img)

        # Print the recognized text
        print(result)
        
        
# Display the image with the license plate region highlighted
cv2.imshow('License Plate Detection', img)
cv2.waitKey(0)
cv2.destroyAllWindows()




error: OpenCV(4.5.4) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1274: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
